In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("xor") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

sc = spark.sparkContext


In [3]:
df = spark.read.option("inferSchema", "true").csv('pitches_preprocessed.csv', header = True)

In [4]:
df = df.select('outs', 'pfx_x', 'pfx_z', 'pitch_num', 'px', 'pz', 'start_speed', 'sz_bot', 'sz_top', 'x0', 'y0',
 'z0', 'batter_id', 'inning', 'p_throws', 'pitcher_id', 'stand', 'latent_pitch_type',
 'count_status','base_status', 'binned_score_difference','latent_next_pitch')

In [5]:
df = df.withColumn('binned_score_difference', df.binned_score_difference +5)

In [6]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols =["latent_pitch_type", "pitch_num", "base_status","binned_score_difference",
                                            "count_status"],
                                 outputCols =["latent_pitch_typeH", "pitch_numH", "base_statusH","binned_score_differenceH"
                                              ,"count_statusH"])
model = encoder.fit(df)
df = model.transform(df)

In [7]:
df = df.select('outs','pfx_x','pfx_z','px','pz','start_speed','sz_bot','sz_top','x0',
               'y0','z0','batter_id','inning','p_throws','pitcher_id','stand','latent_pitch_typeH','pitch_numH',
               'base_statusH','binned_score_differenceH','count_statusH','latent_next_pitch')

In [8]:
def transData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-6]), r[-2], r[-3], r[-4], r[-5], r[-6]]).\
           toDF(['label','features', 'count_statusH','binned_score_differenceH','base_statusH', 'pitch_numH',
                 'latent_pitch_typeH'])

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

data= transData(df)
data.show()

+-----+--------------------+---------------+------------------------+-------------+--------------+------------------+
|label|            features|  count_statusH|binned_score_differenceH| base_statusH|    pitch_numH|latent_pitch_typeH|
+-----+--------------------+---------------+------------------------+-------------+--------------+------------------+
|  0.0|[1.0,6.08,9.83,-0...| (11,[2],[1.0])|          (10,[4],[1.0])|(7,[0],[1.0])|(15,[2],[1.0])|     (9,[0],[1.0])|
|  0.0|[1.0,4.54,12.83,-...| (11,[5],[1.0])|          (10,[4],[1.0])|(7,[0],[1.0])|(15,[3],[1.0])|     (9,[0],[1.0])|
|  7.0|[0.0,-3.71,9.05,-...| (11,[2],[1.0])|              (10,[],[])|(7,[0],[1.0])|(15,[2],[1.0])|     (9,[0],[1.0])|
|  7.0|[0.0,4.87,-6.37,0...| (11,[3],[1.0])|              (10,[],[])|(7,[0],[1.0])|(15,[3],[1.0])|     (9,[7],[1.0])|
|  0.0|[0.0,1.64,-4.12,0...| (11,[8],[1.0])|              (10,[],[])|(7,[0],[1.0])|(15,[4],[1.0])|     (9,[7],[1.0])|
|  6.0|[0.0,-2.47,9.54,-...|(11,[10],[1.0])|            

In [9]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

norm = Normalizer(inputCol='features', outputCol='features_norm', p=1.0)
data = norm.transform(data)

In [10]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['features_norm', 'latent_pitch_typeH','pitch_numH','base_statusH',
                                         'binned_score_differenceH','count_statusH'], outputCol = 'features_fin')

data = assembler.transform(data)

In [11]:
data = data.select('label', 'features_fin')

In [12]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier(labelCol="label", featuresCol="features_fin")
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

In [14]:
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [3, 6, 10]).build()

In [15]:
crossval = CrossValidator(
    estimator=rf,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3)

In [16]:
model = crossval.fit(trainingData)

In [19]:
predictions = model.transform(testData)

In [23]:
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

Accuracy = 0.381832


In [37]:
evaluatortwo = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluatortwo.evaluate(predictions)
print("F1 = %g" % f1)

F1 = 0.300129


In [ ]:
evaluatorthree = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluatorthree.evaluate(predictions)
print("Weighted Precision = %g" % precision)

Weighted Precision = 0.334015


In [20]:
evaluatorfour = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluatorfour.evaluate(predictions)
print("Weighted Recall = %g" % recall)

Weighted Recall = 0.381832


Row(label=0.0, features_fin=SparseVector(68, {0: 0.0, 1: 0.0, 2: 0.0, 3: -0.0, 4: 0.0, 5: 0.0002, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0001, 10: 0.0, 11: 0.2178, 12: 0.0, 14: 0.7818, 16: 1.0, 27: 1.0, 40: 1.0, 51: 1.0, 59: 1.0}))